# Tutorial - Spark in Scala
This notebook is designed to introduce some basic concepts and help get you familiar with using Spark in Scala.  

In this notebook, we will load and explore the mtcars dataset. Specifically, this tutorial covers:

1. Loading data in memory
1. Creating SQLContext
1. Creating Spark DataFrame
1. Group data by columns 
1. Operating on columns
1. Running SQL Queries from a Spark DataFrame


## Loading in a DataFrame
To create a Spark DataFrame we load an external DataFrame, called `mtcars`. This DataFrame includes 32 observations on 11 variables.

[, 1]	mpg	Miles/(US) --> gallon  
[, 2]	cyl	--> Number of cylinders  
[, 3]	disp	--> Displacement (cu.in.)  
[, 4]	hp -->	Gross horsepower  
[, 5]	drat -->	Rear axle ratio  
[, 6]	wt -->	Weight (lb/1000)  
[, 7]	qsec -->	1/4 mile time  
[, 8]	vs -->	V/S  
[, 9]	am -->	Transmission (0 = automatic, 1 = manual)  
[,10]	gear -->	Number of forward gears  
[,11]	carb -->	Number of carburetors  

In [ ]:
import sys.process._
import java.net.URL
import java.io.File

def fileDownloader(url: String, filename: String) = {
    new URL(url) #> new File(filename) !!
}

fileDownloader("https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv", "/resources/mtcars.csv")

## Initialize SQLContext
To work with dataframes we need a SQLContext which is created using `SQLContext(sc)`. SQLContext uses SparkContext which has been already created, named `sc`. 

In [ ]:
import au.com.bytecode.opencsv.CSVParser
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

## Creating Spark DataFrames
With SQLContext and a loaded local DataFrame, we create a Spark DataFrame:

In [ ]:
// Define the schema using a case class.
case class Cars(car: String, mpg: String, cyl: String, disp: String, hp: String, drat: String, wt: String, qsec: String, vs: String, am: String, gear: String, carb: String)

In [ ]:
val csv = sc.textFile("/resources/mtcars.csv")
val headerAndRows = csv.map(line => line.split(",").map(_.trim))
val header = headerAndRows.first
val data = headerAndRows.filter(_(0) != header(0))
val mtcars = data.map(p => Cars(p(0), p(1), p(2), p(3), p(4), p(5), p(6), p(7), p(8), p(9), p(10), p(11))).toDF()
mtcars.printSchema

## Displays the content of the DataFrame 


In [ ]:
mtcars.show(5)

## Selecting columns

In [ ]:
mtcars.select("mpg").show(5)

## Filtering Data
Filter the DataFrame to only retain rows with `mpg` less than 18

In [ ]:
mtcars.filter(mtcars("mpg") < 18).show(5)

## Operating on Columns
SparkR also provides a number of functions that can directly applied to columns for data processing and aggregation. The example below shows the use of basic arithmetic functions to convert lb to metric ton.

In [ ]:
mtcars.withColumn("wtTon", mtcars("wt") * 0.45).show(6)

## Grouping, Aggregation
Spark DataFrames support a number of commonly used functions to aggregate data after grouping. For example we can compute the average weight of cars by their cylinders as shown below:

In [ ]:
import org.apache.spark.sql.functions._
mtcars.groupBy("cyl").agg(avg("wt")).show(5)

In [ ]:
// We can also sort the output from the aggregation to get the most common cars

mtcars.groupBy("cyl").agg(count("wt")).sort($"count(wt)".desc).show(5)

### Running SQL Queries from Spark DataFrames
A Spark DataFrame can also be registered as a temporary table in Spark SQL and registering a DataFrame as a table allows you to run SQL queries over its data. The `sql` function enables applications to run SQL queries programmatically and returns the result as a DataFrame.



In [ ]:
//Register this DataFrame as a table.
mtcars.registerTempTable("cars")

// SQL statements can be run by using the sql methods provided by sqlContext.
val highgearcars = sqlContext.sql("SELECT gear FROM cars WHERE cyl >= 4 AND cyl <= 9")
highgearcars.show(6)

NOTE: This tutorial draws heavily from the original 
[Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html)

## Want to learn more?

### Free courses on [Big Data University](http://bigdatauniversity.com/courses/spark-overview/?utm_source=tutorial-spark-scala&utm_medium=dswb&utm_campaign=bdu):
<a href="https://bigdatauniversity.com/courses/spark-overview-scala-analytics/?utm_source=tutorial-spark-scala&utm_medium=dswb&utm_campaign=bdu"><img src = https://ibm.box.com/shared/static/4w1zym7ek3ujbr9keseck3xi6nqor4iz.png align=left> </a>


#### Complete free [courses on Scala](https://bigdatauniversity.com/learn/scala/?utm_source=tutorial-spark-scala&utm_medium=dswb&utm_campaign=bdu) and earn [IBM badges](https://bigdatauniversity.com/badges/scala-programming-explorer/?utm_source=tutorial-spark-scala&utm_medium=dswb&utm_campaign=bdu):    

<a href="https://bigdatauniversity.com/learn/scala/?utm_source=tutorial-spark-scala&utm_medium=dswb&utm_campaign=bdu"><img src = https://ibm.box.com/shared/static/oxkbzzwgxg9wnr4texux044t42kdnweo.png width=640 align=left> </a>

<h3>Authors:</h3>
<br>
<a href="https://ca.linkedin.com/in/saeedaghabozorgi">
    <div class="teacher-image" style="    float: left;
        width: 115px;
        height: 115px;
        margin-right: 10px;
        margin-bottom: 10px;
        border: 1px solid #CCC;
        padding: 3px;
        border-radius: 3px;
        text-align: center;"><img class="alignnone wp-image-2258 " src="https://ibm.box.com/shared/static/tyd41rlrnmfrrk78jx521eb73fljwvv0.jpg" alt="Saeed Aghabozorgi" width="178" height="178"/>
    </div>
</a>

<h4>Saeed Aghabozorgi</h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<br>

<a href="https://ca.linkedin.com/in/polonglin">
    <div class="teacher-image" style="    float: left;
        width: 115px;
        height: 115px;
        margin-right: 10px;
        margin-bottom: 10px;
        border: 1px solid #CCC;
        padding: 3px;
        border-radius: 3px;
        text-align: center;"><img class="alignnone size-medium wp-image-2177" src="https://ibm.box.com/shared/static/2ygdi03ahcr97df2ofrr6cf8knq4kodd.jpg" alt="Polong Lin" width="300" height="300"/>
    </div>
</a>
<h4>Polong Lin</h4>
<p>
<a href="https://ca.linkedin.com/in/polonglin">Polong Lin</a> is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds a M.Sc. in Cognitive Psychology.</p>

<hr>
Copyright &copy; 2016 [Big Data University](https://bigdatauniversity.com/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​